In [1]:
from keras.utils.data_utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Bidirectional, LSTM
from keras.datasets import imdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
max_features = 20000
maxlen = 250
batch_size = 128
epochs = 3

In [5]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=max_features,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)

In [6]:
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [7]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_vector_length, input_length=maxlen))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 32)           640000    
                                                                 
 dropout (Dropout)           (None, 250, 32)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 250, 64)          16640     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0

In [8]:
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

157/157 [==============================] - 86s 492ms/step - loss: 0.4890 - accuracy: 0.7458 - val_loss: 0.3547 - val_accuracy: 0.8520
Epoch 2/3
157/157 [==============================] - 74s 471ms/step - loss: 0.2523 - accuracy: 0.9033 - val_loss: 0.3161 - val_accuracy: 0.8654
Epoch 3/3
157/157 [==============================] - 75s 477ms/step - loss: 0.1622 - accuracy: 0.9416 - val_loss: 0.3240 - val_accuracy: 0.8654


In [9]:
predictions = model.predict(X_test)

binary_predictions = ['positive' if pred > 0.5 else 'negative' for pred in predictions]

for i in range(10):
    print(f"Predicted: {binary_predictions[i]}, Label: {'positive' if y_test[i] > 0.5 else 'negative'}")

test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

782/782 [==============================] - 46s 57ms/step
Predicted: negative, Label: negative
Predicted: positive, Label: positive
Predicted: positive, Label: positive
Predicted: positive, Label: negative
Predicted: positive, Label: positive
Predicted: positive, Label: positive
Predicted: positive, Label: positive
Predicted: negative, Label: negative
Predicted: positive, Label: negative
Predicted: positive, Label: positive
782/782 [==============================] - 45s 57ms/step - loss: 0.3393 - accuracy: 0.8584
Test Loss: 0.33929336071014404
Test Accuracy: 0.8583999872207642
